In [ ]:
#MPS
import math
import pandas

In [ ]:
def MPS_batch(demand_forecast, Orders, init_inventory, ss, Q_size):
  available_to_promise = []
  MPS = []
  inventory = []
  for i in range(len(demand_forecast)-len(Orders)):
    Orders.append(0)
  #MPS, Inventory
  for i in range(len(demand_forecast)):
    if i == 0:
      if init_inventory-max(Orders[0],demand_forecast[0]) < ss:
        MPS.append(Q_size*(math.ceil((max(Orders[0],demand_forecast[0])-init_inventory+ss)/Q_size)))
      else:
        MPS.append(0)
      inventory.append(init_inventory+MPS[0]-max(Orders[0],demand_forecast[0]))
    else:
      if inventory[i-1]-max(Orders[i],demand_forecast[i]) < ss:
        MPS.append(Q_size*(math.ceil((max(Orders[i],demand_forecast[i])-inventory[i-1]+ss)/Q_size)))
      else:
        MPS.append(0)
      inventory.append(inventory[i-1]+MPS[i]-max(Orders[i],demand_forecast[i]))
  #ATP
  for i in range(len(demand_forecast)):
    if i == 0:
      x=init_inventory+MPS[i]-Orders[i]
      for j in range(i,len(demand_forecast)):
        if MPS[j] == 0:
          x-=Orders[j]
        else:
          break
      available_to_promise.append(x)
    else:
      if MPS[i] == 0:
        available_to_promise.append(0)
      else:
        x=MPS[i]-Orders[i]
        for j in range(i+1,len(demand_forecast)):
          if MPS[j] == 0:
            x-=Orders[j]
          else:
            break
        available_to_promise.append(x)

  data = pandas.DataFrame([demand_forecast, Orders, inventory, available_to_promise, MPS], columns=[i+1 for i in range(len(demand_forecast))], index=["Forecast", "Orders","Project available balace","Available-to-promise","MPS"])

  print("Batch MPS")
  print(data)
  print("on hand:", init_inventory, "ss:",ss, "lot size:",Q_size)
  print("\n")

In [ ]:
def MPS_chase(demand_forecast, Orders, init_inventory, ss):
  global MPS
  available_to_promise = []
  MPS = []
  inventory = []
  for i in range(len(demand_forecast)-len(Orders)):
    Orders.append(0)
  #MPS, Inventory
  for i in range(len(demand_forecast)):
    if i == 0:
      if init_inventory-max(Orders[0],demand_forecast[0]) < ss:
        MPS.append(max(Orders[0],demand_forecast[0])-init_inventory+ss)
      else:
        MPS.append(0)
      inventory.append(init_inventory+MPS[0]-max(Orders[0],demand_forecast[0]))
    else:
      if inventory[i-1]-max(Orders[i],demand_forecast[i]) < ss:
        MPS.append(max(Orders[i],demand_forecast[i])-inventory[i-1]+ss)
      else:
        MPS.append(0)
      inventory.append(inventory[i-1]+MPS[i]-max(Orders[i],demand_forecast[i]))
  #ATP
  for i in range(len(demand_forecast)):
    if i == 0:
      x=init_inventory+MPS[i]-Orders[i]
      for j in range(i,len(demand_forecast)):
        if MPS[j] == 0:
          x-=Orders[j]
        else:
          break
      available_to_promise.append(x)
    else:
      if MPS[i] == 0:
        available_to_promise.append(0)
      else:
        x=MPS[i]-Orders[i]
        for j in range(i+1,len(demand_forecast)):
          if MPS[j] == 0:
            x-=Orders[j]
          else:
            break
        available_to_promise.append(x)

  data = pandas.DataFrame([demand_forecast, Orders, inventory, available_to_promise, MPS], columns=[i+1 for i in range(len(demand_forecast))], index=["Forecast", "Orders","Project available balace","Available-to-promise","MPS"])

  print("Chase MPS")
  print(data)
  print("on hand:", init_inventory, "ss:", ss)
  print("\n")

In [ ]:
demand_forecast=[10,10,10,10,10,15,15,15,15,15,15,15]
Orders=[5,3,2]
init_inventory=10
print("item: Phantom")
MPS_batch(demand_forecast,Orders,init_inventory,10,50)
MPS_chase(demand_forecast,Orders,init_inventory,10)

demand_forecast=[20,20,20,20,20,25,25,25,25,30,30,30]
Orders=[10,7,5,6]
init_inventory=15
print("item: Ghost")
MPS_batch(demand_forecast,Orders,init_inventory,0,50)
MPS_chase(demand_forecast,Orders,init_inventory,10)

demand_forecast=[15,15,15,15,15,20,20,20,20,25,25,25]
Orders=[8,5,3,7]
init_inventory=10
print("item: Cullinan")
MPS_batch(demand_forecast,Orders, init_inventory,0, 50)
MPS_chase(demand_forecast,Orders,init_inventory,10)

item: Phantom
Batch MPS
                          1   2   3   4   5   6   7   8   9   10  11  12
Forecast                  10  10  10  10  10  15  15  15  15  15  15  15
Orders                     5   3   2   0   0   0   0   0   0   0   0   0
Project available balace  50  40  30  20  10  45  30  15  50  35  20  55
Available-to-promise      55   0   0   0   0  50   0   0  50   0   0  50
MPS                       50   0   0   0   0  50   0   0  50   0   0  50
on hand: 10 ss: 10 lot size: 50


Chase MPS
                          1   2   3   4   5   6   7   8   9   10  11  12
Forecast                  10  10  10  10  10  15  15  15  15  15  15  15
Orders                     5   3   2   0   0   0   0   0   0   0   0   0
Project available balace  10  10  10  10  10  10  10  10  10  10  10  10
Available-to-promise      15   7   8  10  10  15  15  15  15  15  15  15
MPS                       10  10  10  10  10  15  15  15  15  15  15  15
on hand: 10 ss: 10


item: Ghost
Batch MPS
             

In [ ]:
def batch_solver(gross_requirment, sr, on_hand, ss, lt, Q_size, print_):
  global p_o_r
  p_o_r=[]
  p_r=[]
  inventory=[]
  for i in range(len(gross_requirment)-len(sr)):
    sr.append(0)
  for i in range(len(gross_requirment)):
    if i==0:
      if on_hand-gross_requirment[0]+sr[0] < ss:
        p_r.append(Q_size*(math.ceil((gross_requirment[0]-on_hand-sr[0]+ss)/Q_size)))
      else:
        p_r.append(0)
      inventory.append(on_hand-gross_requirment[0]+sr[0]+p_r[0])
    else:
      if inventory[i-1]+sr[i]-gross_requirment[i] < ss:
        p_r.append(Q_size*(math.ceil((gross_requirment[i]-inventory[i-1]-sr[i]+ss)/Q_size)))
      else:
        p_r.append(0)
      inventory.append(inventory[i-1]-gross_requirment[i]+sr[i]+p_r[i])
  for i in range(lt):
    p_r.pop(0)
    p_r.append(0)
  p_o_r=p_r

  if print_==True:
    data = pandas.DataFrame([gross_requirment,sr,inventory,p_o_r], columns=[i+1 for i in range(len(gross_requirment))], index=["Gross requirements", "Scheduled receipts", "Project available balance", "Planned order release"])
    print("batch")
    print(data)
    print("on hand:",on_hand, "ss:",ss,"lt:",lt, "lot size:",Q_size)
    print("\n")

In [ ]:
def L4L_solver(gross_requirment, sr, on_hand, ss, lt, print_):
  global p_o_r
  p_o_r=[]
  p_r=[]
  inventory=[]
  for i in range(len(gross_requirment)-len(sr)):
    sr.append(0)
  for i in range(len(gross_requirment)):
    if i==0:
      if on_hand-gross_requirment[0]+sr[0] < ss:
        p_r.append(gross_requirment[0]-on_hand-sr[0]+ss)
      else:
        p_r.append(0)
      inventory.append(on_hand-gross_requirment[0]+sr[i]+p_r[0])
    else:
      if inventory[i-1]+sr[i]-gross_requirment[i] < ss:
        p_r.append(gross_requirment[i]-inventory[i-1]-sr[i]+ss)
      else:
        p_r.append(0)
      inventory.append(inventory[i-1]-gross_requirment[i]+sr[i]+p_r[i])
  for i in range(lt):
    p_r.pop(0)
    p_r.append(0)
  p_o_r=p_r

  if print_==True:
    data = pandas.DataFrame([gross_requirment,sr,inventory,p_o_r], columns=[i+1 for i in range(len(gross_requirment))], index=["Gross requirements", "Scheduled receipts", "Project available balance", "Planned order release"])
    print("L4L")
    print(data)
    print("on hand:",on_hand, "ss:",ss,"lt:",lt)
    print("\n")

In [ ]:
BOM = [["a","","b","",""],
       ["c","","c","","d"],
       ["e","f","e","f",""]]
multi_BOM = [[1,1,1,1,1],
             [1,1,1,1,2],
             [1,2,1,2,1]]
Lt_list={"a":1,"b":1,"c":1,"d":1,"e":2,"f":1}
sr_list={"a":[30,60],"b":[40,90],"c":[100,30],
         "d":[40,20], "e":[20,90],"f":[90,120,30]}
ss_list={"a":5,"b":0,"c":0,"d":0,"e":5,"f":10}
on_hand_list={"a":50,"b":40,"c":35,"d":61,"e":41,"f":32}
MPS_list = {"a":[50,0,0,0,0,50,0,0,50,0,0,50],
            "b":[50,0,0,50,0,50,0,50,0,50,0,50],
            "c":[0,0,0,0,0,0,0,0,0,0,0,0],
            "d":[0,0,0,0,0,0,0,0,0,0,0,0],
            "e":[0,0,0,0,0,0,0,0,0,0,0,0],
            "f":[0,0,0,0,0,0,0,0,0,0,0,0]}
p_o_r_list = {"a":[],"b":[],"c":[],"d":[],"e":[],"f":[]}

In [ ]:
for i in range(len(p_o_r_list)):
  p_o_r_list[i]=[0,0,0,0,0,0,0,0,0,0,0,0]

In [ ]:
#level0
for i in range(len(BOM[0])):
    if BOM[0][i]!="":
      print("Part:",BOM[0][i])
      L4L_solver(MPS_list[BOM[0][i]],sr_list[BOM[0][i]],on_hand_list[BOM[0][i]],ss_list[BOM[0][i]], Lt_list[BOM[0][i]],True)
      p_o_r_list[BOM[0][i]] = p_o_r
    for k in range(len(BOM[0])):
      if BOM[0][i] == '':
        BOM[0][i] = BOM[0][i-1]

#level n
for k in range(1,len(BOM)):
  for i in range(len(BOM[k])):
    if BOM[k][i]!="":
      for j in range(len(MPS_list["a"])):
        MPS_list[BOM[k][i]][j]+=multi_BOM[k][i]*p_o_r_list[BOM[k-1][i]][j]
      if k != len(BOM):
        if i != len(BOM[k]):
          if BOM[k][i] not in BOM[k][i+1:]:
            if BOM[k][i] not in BOM[k+1:]:
              print("Part:",BOM[k][i])
              L4L_solver(MPS_list[BOM[k][i]],sr_list[BOM[k][i]],on_hand_list[BOM[k][i]],ss_list[BOM[k][i]],Lt_list[BOM[k][i]],True)
              p_o_r_list[BOM[k][i]] = p_o_r
        else:
          if BOM[k][i] not in BOM[k+1:]:
            print("Part:",BOM[k][i])
            L4L_solver(MPS_list[BOM[k][i]],sr_list[BOM[k][i]],on_hand_list[BOM[k][i]],ss_list[BOM[k][i]],Lt_list[BOM[k][i]],True)
            p_o_r_list[BOM[k][i]] = p_o_r
      else:
        if i != len(BOM[k]):
          if BOM[k][i] not in BOM[k][i+1]:
            print("Part:",BOM[k][i])
            L4L_solver(MPS_list[BOM[k][i]],sr_list[BOM[k][i]],on_hand_list[BOM[k][i]],ss_list[BOM[k][i]],Lt_list[BOM[k][i]],True)
            p_o_r_list[BOM[k][i]] = p_o_r
        else:
          print("Part:",BOM[k][i])
          L4L_solver(MPS_list[BOM[k][i]],sr_list[BOM[k][i]],on_hand_list[BOM[k][i]],ss_list[BOM[k][i]],Lt_list[BOM[k][i]],True)
          p_o_r_list[BOM[k][i]] = p_o_r

    if BOM[k][i] == "":
      BOM[k][i] = BOM[k][i-1]

Part: a
L4L
                           1   2   3   4   5   6   7   8   9   10  11  12
Gross requirements         50   0   0   0   0  50   0   0  50   0   0  50
Scheduled receipts         30  60   0   0   0   0   0   0   0   0   0   0
Project available balance  30  90  90  90  90  40  40  40   5   5   5   5
Planned order release       0   0   0   0   0   0   0  15   0   0  50   0
on hand: 50 ss: 5 lt: 1


Part: b
L4L
                           1    2    3   4   5   6   7   8   9   10  11  12
Gross requirements         50    0    0  50   0  50   0  50   0  50   0  50
Scheduled receipts         40   90    0   0   0   0   0   0   0   0   0   0
Project available balance  30  120  120  70  70  20  20   0   0   0   0   0
Planned order release       0    0    0   0   0   0  30   0  50   0  50   0
on hand: 40 ss: 0 lt: 1


Part: c
L4L
                            1    2    3    4    5    6    7    8   9   10  \
Gross requirements           0    0    0    0    0    0   30   15  50   0   
Schedule

In [ ]:
len(MPS_list["a"])


12